In [16]:
#!/usr/bin/env python3
"""
Shows how to toss a capsule to a container.
"""
from mujoco_py import load_model_from_path, MjSim, MjViewer
import os
import torch
import torch.nn as nn
from random import *
import mujoco
from mujoco import *

# xml_file = load_model_from_path("./xmls/cartpole.xml")
sim = MjSim(xml_file)

viewer = MjViewer(sim)
sim_state = sim.get_state()

while True:
    #print(sim_state)
    sim.set_state(sim_state)

    for i in range(1):
        
        if i<30:
            sim.data.ctrl[:] = 0.1
            #sim.data.ctrl[0:] = -1.0
            #sim.data.ctrl[1:] = 0.0
        else:
            sim.data.ctrl[:] = 0.1
            #sim.data.ctrl[0:] = 0.0
            #sim.data.ctrl[1:] = -1.0
        sim.step()
        #print(sim.get_state())
        viewer.render()

    if os.getenv('TESTING') is not None:
        break



Creating window glfw


SystemExit: 0

/home/dnclab/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from mujoco_py import load_model_from_path, MjSim, MjViewer
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from random import *

import math

# import gym
import numpy as np

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from common.multiprocessing_env import SubprocVecEnv

num_sims = 16
xml_file = load_model_from_path("./xmls/cartpole.xml")

def make_sim():
    def _thunk():
        sim = MjSim(xml_file)
        return sim
    return _thunk

sims = [make_sim() for i in range(num_sims)]
sims = SubprocVecEnv(sims)

sim = MjSim(xml_file)

sim_init_state = sim.get_state()
# viewer = MjViewer(sim)

In [15]:
print(sim.data.ctrl)
curr = sim.get_state()
print(curr)
print(curr.qpos)
print(curr.qvel)
print(sim)

[0.]
MjSimState(time=0.0, qpos=array([0., 0.]), qvel=array([0., 0.]), act=None, udd_state={})
[0. 0.]
[0. 0.]


In [ ]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20, 5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    state = sim.set_state(sim_init_state)
    if vis: viewer.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state)